In [1]:
import sys
sys.path.append("../src")

In [13]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.utils.dtcc_swaps_utils import datetime_to_ql_date, tenor_to_ql_period, build_ql_piecewise_curves, swap_spreads_term_structure, tenor_to_years
from datetime import datetime
import pandas.tseries.offsets as offsets
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from scipy.optimize import brentq
import scipy.interpolate
import rateslib as rl
import QuantLib as ql
import plotly.graph_objects as go
from termcolor import colored
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import statsmodels.api as sm
from typing import Dict, List, Optional, Tuple
import urllib.parse
import requests
import pickle
import os
import shelve
from pandas.tseries.offsets import CustomBusinessDay, BDay, BMonthEnd
from pandas.tseries.holiday import USFederalHolidayCalendar

from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
print(env_path)
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import matplotlib.pylab as pylab
params = {
    "legend.fontsize": "x-large",
    "axes.labelsize": "x-large",
    "axes.titlesize": "x-large",
    "xtick.labelsize": "x-large",
    "ytick.labelsize": "x-large",
}
pylab.rcParams.update(params)
sns.set(style="whitegrid", palette="dark")
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

c:\Users\chris\Project Bond King\Curvy-CUSIPs\notebooks\../.env
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
curve_data_fetcher = CurveDataFetcher(use_ust_issue_date=True, fred_api_key=config["FRED_API_KEY"], error_verbose=True, debug_verbose=True)

2024-11-24 13:07:57,345 - CurvyCUSIPs.DataFetcher.base - DEBUG - UST Auctions - Number of Links to Fetch: 2
2024-11-24 13:07:57,346 - CurvyCUSIPs.DataFetcher.base - DEBUG - UST Auctions - Links: ['https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=1&page[size]=10000', 'https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?page[number]=2&page[size]=10000']


In [4]:
curve_data_fetcher.dtcc_sdr_fetcher.setup_s490_nyclose_db(r"C:\Users\chris\Project Bond King\ql_swap_curve_objs\nyclose_s490_usd_sofr_curve_dtcc_sdr_master")

In [25]:
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 11, 21)

sofr_ois_df = curve_data_fetcher.dtcc_sdr_fetcher.s490_nyclose_term_structure_ts(start_date=start_date, end_date=end_date)
ct_df = curve_data_fetcher.fedinvest_data_fetcher.get_historical_ct_yields(start_date=start_date, end_date=end_date)
swap_spread_df = swap_spreads_term_structure(swaps_term_structure_ts_df=sofr_ois_df.set_index("Date"), cash_term_structure_ts_df=ct_df.set_index("Date"))
swap_spread_df

,3M,6M,12M,2Y,3Y,5Y,7Y,10Y,20Y,30Y
Date,,,,,,,,,,
2024-01-02,3.560976,-0.817523,1.242338,-22.974293,-26.906464,-34.583342,-41.755999,-42.017960,-72.363095,-72.343242
2024-01-03,2.888777,0.733992,3.453715,-17.657599,-24.323940,-30.422108,-37.468890,-39.424314,-68.119296,-68.668287
2024-01-04,9.366469,-2.135386,3.330176,-16.339765,-23.278453,-30.941401,-38.159529,-37.695671,-67.248561,-67.943153
2024-01-05,1.179448,0.743035,1.424558,-16.433474,-24.921634,-30.644154,-37.778340,-39.293151,-69.177305,-70.811522
2024-01-08,1.357440,0.479334,3.742113,-16.420034,-21.654324,-27.086639,-34.850127,-36.121186,-65.340166,-66.934721
...,...,...,...,...,...,...,...,...,...,...
2024-11-15,5.949413,3.164078,-3.940376,-20.852565,-25.666682,-35.897817,-44.556284,-52.383263,-76.398389,-83.898634
2024-11-18,6.454599,1.875199,-4.902459,-23.720799,-24.389074,-34.835995,-44.205518,-50.863227,-77.704359,-84.306375
2024-11-19,6.304771,0.365614,-5.016004,-19.641406,-24.674271,-34.631383,-45.250967,-48.907283,-74.975996,-82.535749


In [28]:
as_of_dates = [datetime(2024, 1, 2), datetime(2024, 11, 1), datetime(2024, 11, 8), datetime(2024, 11, 15), datetime(2024, 11, 21)]

fig = go.Figure()
for curr in as_of_dates:
    try:
        spreads_dict = swap_spread_df[swap_spread_df.index == curr].to_dict("records")[0]
        fig.add_trace(
            go.Scatter(x=[tenor_to_years(tenor) for tenor in spreads_dict.keys()], y=list(spreads_dict.values()), mode="lines", name=f"{curr.date()}")
        )
    except Exception as e:
        print(f"{curr} NOT IN DB", e)

fig.update_layout(title="SOFR OIS Swap Spreads NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()

In [21]:
as_of_dates = [datetime(2024, 11, 1), datetime(2024, 11, 22)]

fig = go.Figure()
for curr in as_of_dates:
    try:
        str_ts = str(int(curr.timestamp()))
        ohlc_df = pd.DataFrame(curve_data_fetcher.dtcc_sdr_fetcher.s490_nyclose_db.get(str_ts)["ohlc"])
        ohlc_df["Expiry"] = pd.to_numeric(ohlc_df["Expiry"], errors="coerce")
        ohlc_df["Expiry"] = pd.to_datetime(ohlc_df["Expiry"], errors="coerce", unit="s")
        fig.add_trace(go.Scatter(x=((ohlc_df["Expiry"] - curr).dt.days / 360), y=ohlc_df["Close"] * 100, mode="lines", name=f"{curr.date()}"))
    except:
        print(f"{curr} NOT IN DB")

fig.update_layout(title="SOFR OIS NY Close Curve", xaxis_title="Tenor", yaxis_title="Fixed Rate", template="plotly_dark", height=1000)
fig.update_xaxes(
    showgrid=True,
    showspikes=True,
    spikecolor="white",
    spikesnap="cursor",
    spikemode="across",
)
fig.update_yaxes(showgrid=True, showspikes=True, spikecolor="white", spikesnap="cursor", spikethickness=0.5)
fig.show()